In [3]:
import pandas as pd
import numpy as np
import utils as ut # esta librería tiene funciones para poder obtener un procesamiento del <T,H>
import spacy
import mutual_info as mi
import time
from scipy.stats import wasserstein_distance
import sys
from math import floor

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
import conceptnet_lite
conceptnet_lite.connect("../OPENAI/data/conceptnet.db")
from conceptnet_lite import Label, edges_for, edges_between

In [5]:
def obtener_distancia(texto_v,hipotesis_v,texto_t,texto_h,b_col,b_index):
    lista_l=[]
    for i in range(len(texto_t)):
        lista=[]
        for j in range(len(texto_h)):
            lista.append(np.linalg.norm(texto_v[i] - hipotesis_v[j]))#*wasserstein_distance(texto_2[i],hipotesis_2[j]))
        lista_l.append(lista)
    df_distEuc=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    df_distEuc=df_distEuc.drop(b_col[1:],axis=1)
    df_distEuc=df_distEuc.drop(b_index[1:],axis=0)
    return df_distEuc

In [6]:
def wasserstein_mutual_inf(texto_v,hipotesis_v,texto_t,texto_h):  
    lista_l=[]
    lista_muinfor=[]   
    for i in range(len(texto_t)):
        lista=[]
        lista_mu=[]
        for j in range(len(texto_h)):
            lista.append(wasserstein_distance(texto_v[i],hipotesis_v[j]))
            lista_mu.append(mi.mutual_information_2d(np.array(texto_v[i]),np.array(hipotesis_v[j])))
        lista_l.append(lista)
        lista_muinfor.append(lista_mu)
    DFmearth=pd.DataFrame(lista_l,index=texto_t,columns=texto_h)
    DFmutual_inf=pd.DataFrame(lista_muinfor,index=texto_t,columns=texto_h)
    return DFmearth,DFmutual_inf

In [7]:
def entropia(X):
    """Devuelve el valor de entropia de una muestra de datos""" 
    probs = [np.mean(X == valor) for valor in set(X)]
    return round(sum(-p * np.log2(p) for p in probs), 3)

In [8]:
relaciones_generales=["related_to","is_a","etymologically_related_to","manner_of","has_a","derived_from","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent"]
relaciones_especificas=["is_a","manner_of","has_a","derived_from","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent"]

In [9]:
def bag_of_synonyms(word):
    sinonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name == "synonym":
                if word== e.start.text:
                    sinonimos.add(e.end.text)
                elif word== e.end.text:
                    sinonimos.add(e.start.text)
    except:
        pass
    sinonimos.add(word)
    return sinonimos

def bag_of_antonyms(word):
    antonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in ["antonym","distinc_from"]:
                if word== e.start.text:
                    antonimos.add(e.end.text)
                elif word== e.end.text:
                    antonimos.add(e.start.text)
    except:
        pass
    return antonimos

def bag_of_hyperonyms(word):
    hiperonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones_generales:
                if word== e.start.text:
                    hiperonimos.add(e.end.text)
    except:
        pass
    return hiperonimos

def bag_of_hyponyms(word):
    hiponimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones_especificas:
                if word== e.end.text:
                    hiponimos.add(e.start.text)
                    #print(e.relation.name,e.start.text)
    except:
        pass
    return hiponimos

In [10]:
def jaro_distanceDavid2(s1, s2,sinT,sinH,HipT,hipH) :
    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
    matchC = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiph HiperT",s2[j],s1[i],hipH[j],HipT[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(match)
    # If there is no match 
    if (match == 0) :
        return 0.0; 
    return match/len2 ,(len2-match-matchC)/len2; 


In [11]:
def jaro_distance(s1, s2,sinT,sinH,HipT,hipH) :
    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ); 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1):
 
        # Check if there is any matches
        for j in range(max(0, i - max_dist), 
                       min(len2, i + max_dist + 1)):
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiponimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiperonimos3",s2[j],s1[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            
    print(hash_s1)
    print(hash_s2)
    print(match)
    # If there is no match 
    if (match == 0) :
        return 0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1    
    t /= 2; 
    #Return the Jaro Similarity 
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0; 

In [12]:
nlp = spacy.load("en_core_web_md") # modelo de nlp

#ut.load_vectors_in_lang(nlp,"../OPENAI/data/glove.840B.300d.txt") # carga de vectores en nlp.wv
ut.load_vectors_in_lang(nlp,"./data/numberbatch-en-17.04b.txt") # carga de vectores en nlp.wv


In [13]:

#prueba=pd.read_csv("data/DEV/pruebaDEV.csv")
#prueba=pd.read_csv("../OPENAI/data/"+sys.argv[1])

#textos = prueba["sentence1"].to_list()       # almacenamiento en listas
#hipotesis = prueba["sentence2"].to_list()
textos=["A man dressed in a red shirt and black tie stands up at a wedding reception to make a speech.","Fishermen using poison sodium cyanide have helped destroy estimated 70 reefs","A woman finishing a marathon race.","Zapatero visited the following cities in four days: Brasilia, São Paulo, Buenos Aires and Santiago de Chile. According to official sources these visits are the last part of the project he began at the EU-Latin American Summit in Guadalajara, Mexico and pursued in the Ibero-American meeting in Costa Rica in November.","The child is hanging upside down with his legs over a pole.","Someone is falling off a horse","As late as 1799, priests were still being imprisoned or deported to penal colonies and persecution only worsened after the French army led by General Louis Alexandre Berthier captured Rome and imprisoned Pope Pius VI, who would die in captivity in Valence, Drôme, France in August of 1799."]
hipotesis= ["a guy in a red top and tie makes a speech","Cyanide fishing linked destruction area reefs","The woman is not going to finish the race.","Zapatero participated in the Ibero-American meeting in Costa Rica.","There is an adult upside down.","The man is not knocked off of a horse","Alexandre Berthier died in 1799."]


In [14]:
tags=[ 'acl','acomp','advcl','advmod','amod','appos','ccomp','complm','compound','conj','infmod','meta','neg',
 'nmod','nn','npadvmod','nounmod','npmod','num','number','nummod','partmod','pcomp','poss','possessive',
 #'prep',
 'quantmod', 'rcmod', 'relcl', 'xcomp', 'adc', 'avc', 'mnr', 'mo', 'ng', 'nmc']

In [15]:
def representacion_entidades(nlp,texto):
    doc = nlp(texto.lower())
    dir_sust=dict()
    palabras=[]
    for token in doc:
        if token.dep_ in tags or "mod" in token.dep_ or "comp" in token.dep_ or "neg" in token.dep_:
            #print(token.text, token.lemma_, token.pos_,token.dep_,token.head.text,token.head.lemma_, token.head.pos_,
            #    [child for child in token.children])
            #sustantivos.append(token.head.lemma_)
            if token.head.lemma_ in dir_sust:
                if dir_sust[token.head.lemma_]=="NA":
                    dir_sust[token.head.lemma_]=token.lemma_
                    if token.head not in palabras:
                        palabras.append(token.head)
                else:
                    dir_sust[token.head.lemma_]=dir_sust[token.head.lemma_]+","+token.lemma_
                    if token.head not in palabras:
                        palabras.append(token.head)
            else:
                dir_sust[token.head.lemma_]=token.lemma_
                if token.head not in palabras:
                    palabras.append(token.head)
        elif token.pos_ in ["NOUN"]:
            #print(token.text,token.lemma_, token.pos_,token.dep_)
            #sustantivos.append(token.lemma_)
            if token.lemma_ not in dir_sust:
                dir_sust[token.lemma_]="NA"
                if token not in palabras:
                    palabras.append(token)
        elif token.pos_ in ["VERB"]:
            #print(token.text, token.lemma_,token.pos_,token.dep_)
            #sustantivos.append(token.lemma_)
            if token.lemma_ not in dir_sust:
                dir_sust[token.lemma_]="NA"
                if token not in palabras:
                    palabras.append(token)
    return dir_sust,palabras

In [16]:
representacion_entidades(nlp,"There is an adult upside down.")

({'be': 'down', 'upside': 'adult', 'down': 'upside'}, [is, upside, down])

In [17]:
def representacion2(nlp,texto):
    doc = nlp(texto.lower())
    dir_sust=dict()
    palabras=[]
    noun_phrase= [chunk.lemma_ for chunk in doc.noun_chunks]
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    for chunk in noun_phrase:
        doc = nlp(chunk)
        nue=[]
        for token in doc:
            if token.pos_!="DET":
                nue.append(token.lemma_)
        if len(nue)>1:
            dir_sust[nue[-1]]=','.join(nue[:-1])
        else:
            dir_sust[nue[-1]]="NA"
    # for chunk in doc.noun_chunks:
    #     if chunk.root.lemma_!=chunk.lemma_:
    #         dir_sust[chunk.root.lemma_]=','.join(chunk.lemma_.split()[:-1])
    #     else:
    #         dir_sust[chunk.root.lemma_]="NA"
    for v in verbs:
        dir_sust[v]="NA"
    #representacion_entidades(nlp,texto)
    palabras.extend([chunk.root.lemma_ for chunk in doc.noun_chunks])
    palabras.extend(verbs)
    return dir_sust,palabras

def relacion_entailment(wt,wh):
    try:
        concepts_wt = Label.get(text=wt, language='en').concepts
        concepts_wh = Label.get(text=wh, language='en').concepts
        for e in edges_between(concepts_wt, concepts_wh):
            if wt == e.start.text and e.relation.name in relaciones_generales:
                print(e.start.text, "-", e.end.text, "|", e.relation.name,e)
                return True
    except:
        pass
    return False
    

In [20]:
def negacion(nlp,texto):
    doc = nlp(texto.lower())
    for token in doc:
        if(token.dep_=="neg"):
            return 1, token.head.lemma_
    return 0,""

In [21]:
def relacion_noentailment(wt,wh):
    try:
        concepts_wt = Label.get(text=wt, language='en').concepts
        concepts_wh = Label.get(text=wh, language='en').concepts
        for e in edges_between(concepts_wt, concepts_wh):
            if wt == e.start.text and e.relation.name in ["distinct_from","antonym"]:
                print(e.start.text, "-", e.end.text, "|", e.relation.name,e)
                return True
    except:
        pass
    return False

In [24]:
new_data = {'sumas' : [], 'distancias' : [], 'entropia_total' : [],'entropias' : [],'mutinf' : [], 
            'mearts' : [], 'max_info' : [],  'list_comp' : [], 'diferencias' :[], 'list_incomp':[],
            'list_M' : [], 'list_m' : [], 'list_T' : [], 'Jaro-Winkler_rit':[],
            'negT' : [], 'verbT' : [], 'negH' : [], 'verbH':[], 
            'clases' : []}

In [25]:
inicio = time.time()
for i in range(len(textos)):
    print(i)
    print(textos[i])
    #r_t,t_clean_m=representacion_entidades(nlp,textos[i])
    r_t,t_clean_m=representacion2(nlp,textos[i])
    neg_t,negadat=negacion(nlp,textos[i])
    new_data['negT'].append(neg_t)
    new_data['verbT'].append(negadat)
    #print(list(r_t.keys()))
    for clave in r_t.keys():
        print("texto",clave,r_t[clave])
    print(hipotesis[i])
    #r_h,h_clean_m = representacion_entidades(nlp,hipotesis[i])
    r_h,h_clean_m = representacion2(nlp,hipotesis[i])
    neg_h,negadah=negacion(nlp,hipotesis[i])
    new_data['negH'].append(neg_h)
    new_data['verbH'].append(negadah)
    for clave in r_h.keys():
        print("hipotesis",clave,r_h[clave])
        
    #print(list(r_h.keys()))
    # t_clean=' '.join(list(r_t.keys()))
    # h_clean=' '.join(list(r_h.keys()))
    t_clean=list(r_t.keys())
    h_clean=list(r_h.keys())
    print("clean")
    print(t_clean)
    print(h_clean)
    # t_clean_m=ut.get_words(textos[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    # h_clean_m=ut.get_words(hipotesis[i],nlp,pos_to_remove=['PUNCT'], normed=True,lemmatize=False)
    # print("get_words")
    # print("clean_m")
    # print(t_clean_m)
    # print(h_clean_m)
    # t_clean_m=ut.reform_sentence2(t_clean_m,nlp)
    # h_clean_m=ut.reform_sentence2(h_clean_m,nlp)
    # print("reform2")
    # print(t_clean_m)
    # print(h_clean_m)
    # t_clean=ut.get_words_rep(t_clean_m,nlp,pos_to_remove=[""],normed=True,lemmatize=False)
    # h_clean=ut.get_words_rep(h_clean_m,nlp,pos_to_remove=[""],normed=True,lemmatize=False)
    # print("t_clean")
    # print(t_clean)
    # print(h_clean)
    # print("clean_m")
    # print(t_clean_m)
    # print(h_clean_m)
    t_vectors=ut.get_matrix_rep2(t_clean, nlp, normed=False)
    h_vectors=ut.get_matrix_rep2(h_clean, nlp, normed=False)
    t_vectors_n=ut.get_matrix_rep2(t_clean, nlp, normed=True)
    h_vectors_n=ut.get_matrix_rep2(h_clean, nlp, normed=True)
    
    # print(t_vectors)
    # print(h_vectors)
    # print(t_clean_m,h_clean_m)
    
    # s1=t_clean_m.split()
    # s2=h_clean_m.split()
    t_lem=ut.get_lemmas_(textos[i],nlp)
    h_lem=ut.get_lemmas_(hipotesis[i],nlp)
    s1=t_lem
    s2=h_lem
    
    sinT=[]
    antT=[]
    HipT=[]
    sinH=[]
    antH=[]
    HipH=[]
    hipH=[]
    
    # # encontrar bolsa de sinonimos de cada token
    for t in s1:
        sinT.append(bag_of_synonyms(t))
        HipT.append(bag_of_hyperonyms(t))

    for h in s2:
        sinH.append(bag_of_synonyms(h))
        hipH.append(bag_of_hyponyms(h))
    print(t_lem)
    print(h_lem)
    tp1=jaro_distance(t_lem, h_lem,sinT,sinH,HipT,hipH)
    new_data['Jaro-Winkler_rit'].append(tp1)
    # #new_data['c_estructura'].append(tp2)
    #new_data['nomatch'].append(tp3)
    
    #tp1,tp2=jaro_distance_relacionadas(t_lem, h_lem,sinT,HipT,sinH,hipH)
    #new_data['Jaro-Winkler_relacionadas'].append(tp1)
    #new_data['c2_estructura'].append(tp2)

    #tp1,tp2=jaro_distance_contra(t_lem, h_lem,antT,HipT,antH,HipH)
    #new_data['Jaro-Winkler_contra'].append(tp1)
    #new_data['c1_estructura'].append(tp2)

    # # Obtencion de matriz de alineamiento, matriz de move earth y mutual information
    ma=np.dot(t_vectors_n,h_vectors_n.T)
    #print(t_clean,h_clean)
    #print(len(t_vectors_n),len(h_vectors_n),len(t_clean),len(h_clean))
    m_earth,m_mi=wasserstein_mutual_inf(t_vectors_n,h_vectors_n,t_clean,h_clean)
    ma=pd.DataFrame(ma,index=t_clean,columns=h_clean)
    #print(ma)

    # # Calculamos la entropia inicial de la matriz de distancias coseno sobre tokens de T y H
    new_data['entropia_total'].append(entropia(ma.round(1).values.flatten())) 

    # ###### BORRADO DE COSAS QUE NO OCUPO, SOLO NOS QUEDAMOS CON INFORMACIÓN DE TIPOS DE PALABRA: NOUN, VERB, ADJ Y ADV
    # # TAMBIÉN OMITIMOS EL VERBO BE DEBIDO A QUE POR LO REGULAR SE UTILIZA COMO AUXILIAR Y ES UN VERBO COPULATIVO
    # # sirve para construir la llamada predicación nominal del sujeto de una oración: 
    # # #el sujeto se une con este verbo a un complemento obligatorio llamado atributo que por lo general determina 
    # # alguna propiedad, estado o equivalencia del mismo, por ejemplo: "Este plato es bueno". "Juan está casado".
    c_compatibilidad=0
    c_incompatibilidad=0
    # c_rel_concep=0
    b_col=[0]

    new_data['list_T'].append(ma.shape[0])
    new_data['list_M'].append(ma.shape[1])
    
    # # #PARA REVISAR SI EXISTEN RELACIONES DE SIMILITUD SEMÁNTICA A TRAVÉS DEL USO DE CONCEPNET
    
    n_index = ma.shape[0]
    n_columns = ma.shape[1]
    pasada=0
    #print(ma.index,ma.columns)
    combinaciones=[]

    iguales=set(ma.index).intersection(set(ma.columns))

    while n_columns>0 and pasada<4:
        borrar=[]
        a = ma.idxmax().values
        b = ma.columns
        for j in range(len(a)):
            #print(a[j]+"_"+b[j] not in combinaciones,a[j]+"_"+b[j],combinaciones)
            if a[j]+"_"+b[j] not in combinaciones:
                combinaciones.append(a[j]+"_"+b[j])
                if(relacion_noentailment(a[j],b[j])):
                    c_incompatibilidad+=1
                elif a[j]==b[j]: ## Si son palabras identicas entonces no hacemos más proceso
                    print(a[j],b[j],True,"misma palabra")
                    if 'NA'!=r_h[b[j]]:
                        for mod_h in r_h[b[j]].split(","):
                            for mod_t in r_t[a[j]].split(","):
                                if(relacion_entailment(mod_t,mod_h)):
                                    print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                                    borrar.append(b[j])
                                    c_compatibilidad+=1
                                elif(mod_t==mod_h):
                                    print(mod_t,mod_h,"la misma")
                                    borrar.append(b[j])
                                    c_compatibilidad+=1
                    else:
                        borrar.append(b[j])
                        c_compatibilidad+=1
                else:# en otro caso buscamos una relación de generalidad
                    print(r_t[a[j]].split(","))
                    print(r_h[b[j]].split(","))
                    r_e = relacion_entailment(a[j],b[j])
                    if r_e:
                        print(a[j],b[j], r_e)
                        print("checar atributos")
                        if 'NA' !=r_h[b[j]]:
                            print("lo que tiene:",b[j],r_h[b[j]])
                            for mod_h in r_h[b[j]].split(","):
                                if 'NA'!=r_t[a[j]]:
                                    for mod_t in r_t[a[j]].split(","):
                                        print(mod_h,mod_t)
                                        if mod_h==mod_t:
                                            print(mod_t,mod_h,True)
                                            borrar.append(b[j])
                                            c_compatibilidad+=1
                                        else:
                                            if(relacion_entailment(mod_t,mod_h)):
                                                print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                                                borrar.append(b[j])
                                                c_compatibilidad+=1
                        else:
                            borrar.append(b[j])
                            c_compatibilidad+=1
                    else:
                        print("Proceso de conjuntos")
                        sin1=bag_of_synonyms(a[j])
                        sin2=bag_of_synonyms(b[j])
                        if len(sin1.intersection(sin2))>0:
                            print(a[j],b[j],"conjuntos de sinonimos",sin1.intersection(sin2))
                            print("checar atributos")
                            if 'NA'!=r_h[b[j]]:
                                print("lo que tiene:",b[j],r_h[b[j]])
                                for mod_h in r_h[b[j]].split(","):
                                    if 'NA'!=r_t[a[j]]:
                                        for mod_t in r_t[a[j]].split(","):
                                            print(mod_h,mod_t)
                                            if mod_h==mod_t:
                                                print(mod_t,mod_h,True)
                                                borrar.append(b[j])
                                                c_compatibilidad+=1
                                            else:
                                                if(relacion_entailment(mod_t,mod_h)):
                                                    print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                                                    borrar.append(b[j])
                                                    c_compatibilidad+=1
                            else:
                                borrar.append(b[j])
                                c_compatibilidad+=1
                        else:
                            Hip1=set()
                            for e in list(sin1):
                                Hip1=Hip1.union(bag_of_hyperonyms(e))
                            if len(Hip1.intersection(sin2))>0:
                                print(a[j],b[j],"Hiperonimos y sinonimos",Hip1.intersection(sin2))
                                print("checar atributos")
                                if 'NA'!=r_h[b[j]]:
                                    print("lo que tiene:",b[j],r_h[b[j]])
                                    for mod_h in r_h[b[j]].split(","):
                                        if 'NA'!=r_t[a[j]]:
                                            for mod_t in r_t[a[j]].split(","):
                                                print(mod_h,mod_t)
                                                if mod_h==mod_t:
                                                    print(mod_t,mod_h,True)
                                                    borrar.append(b[j])
                                                    c_compatibilidad+=1
                                                else:
                                                    if(relacion_entailment(mod_t,mod_h)):
                                                        print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                                                        borrar.append(b[j])
                                                        c_compatibilidad+=1
                                else:
                                    borrar.append(b[j])
                                    c_compatibilidad+=1
                            else:
                                hip2=set()
                                for e in list(sin2):
                                    hip2=hip2.union(bag_of_hyponyms(e))
                                if len(sin1.intersection(hip2))>0:
                                    print(a[j],b[j],"Sinonimos e hiponimos",sin1.intersection(hip2))
                                    print("checar atributos")
                                    if 'NA'!=r_h[b[j]]:
                                        for mod_h in r_h[b[j]].split(","):
                                            if 'NA'!=r_t[a[j]]:
                                                for mod_t in r_t[a[j]].split(","):
                                                    #print(mod_h,mod_t)
                                                    if mod_h==mod_t:
                                                        #print(mod_t,mod_h,True)
                                                        borrar.append(b[j])
                                                        c_compatibilidad+=1
                                                    else:
                                                        if(relacion_entailment(mod_t,mod_h)):
                                                            #print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                                                            borrar.append(b[j])
                                                            c_compatibilidad+=1
                                    else:
                                        borrar.append(b[j])
                                        c_compatibilidad+=1
            #else:
                #print(a[j],b[j],"ya se habia revisado")
        for equal in iguales:
            if equal+"_"+equal not in combinaciones:
                if 'NA'!=r_h[equal]:
                    for mod_h in r_h[equal].split(","):
                        for mod_t in r_t[equal].split(","):
                            #print(mod_t,mod_h,relacion_entailment(mod_t,mod_h))
                            if(relacion_entailment(mod_t,mod_h)):
                                borrar.append(equal)
                                c_compatibilidad+=1
                else:
                    borrar.append(str(b[j]))
                    c_compatibilidad+=1
        pasada+=1
        ma = ma.drop(borrar,axis=1)
        m_earth = m_earth.drop(borrar,axis=1)
        m_mi = m_mi.drop(borrar,axis=1)
        n_columns = ma.shape[1]
        b_col.extend(borrar)
    b_index=[0]
    
    # #   ALMACENAMIENTO DE TODA LA INFORMACIÓN PROCESADA DE CARACTERÍSTICAS
    m_distancia = obtener_distancia(t_vectors,h_vectors,t_clean,h_clean,b_col,b_index)
    
    new_data['distancias'].append(m_distancia.max().sum()) #cambie de maximas a sumas
    m_earth=m_earth*m_distancia
    if ma.shape[1]==0:
        new_data['entropias'].append(0)
        new_data['max_info'].append(0)
        new_data['sumas'].append(0)
        new_data['mearts'].append(0)
        new_data['mutinf'].append(0)
        new_data['diferencias'].append(0)
    else:
        new_data['entropias'].append(entropia(ma.round(1).values.flatten()))
        new_data['max_info'].append(ma.max().sum()/(ma.shape[1]))# 
        new_data['sumas'].append(ma.sum().sum()/(ma.shape[1]))# 
        new_data['mearts'].append(m_earth.min().sum()/(ma.shape[1]))# 
        new_data['mutinf'].append(m_mi.max().sum()/(ma.shape[1]))# 
        new_data['diferencias'].append(len(ma.columns)/len(ma.index))

    new_data['list_comp'].append(c_compatibilidad)
    new_data['list_incomp'].append(c_incompatibilidad)
    # new_data['list_rel_con'].append(c_rel_concep)
    # new_data['list_relaciones'].append(parejas)
    # new_data['listas_malignf'].append(ma)
    new_data['list_m'].append(ma.shape[1])
    #new_data['clases'].append(prueba.at[i,"gold_label"])
    new_data['clases'].append(1)
    print(ma)
fin = time.time()
print("Tiempo que se llevo:",round(fin-inicio,2)," segundos")

0
A man dressed in a red shirt and black tie stands up at a wedding reception to make a speech.
texto man NA
texto shirt red
texto tie black
texto reception wedding
texto speech NA
texto dress NA
texto stand NA
texto make NA
a guy in a red top and tie makes a speech
hipotesis guy NA
hipotesis top red
hipotesis tie NA
hipotesis speech NA
hipotesis make NA
clean
['man', 'shirt', 'tie', 'reception', 'speech', 'dress', 'stand', 'make']
['guy', 'top', 'tie', 'speech', 'make']
['man', 'dress', 'red', 'shirt', 'black', 'tie', 'stand', 'wedding', 'reception', 'make', 'speech']
['guy', 'red', 'top', 'tie', 'make', 'speech']
man guy
hiperonimos sobre sinonimos guy man
dress guy
dress red
hiperonimos sobre sinonimos red dress
red guy
red red
red top
hiperonimos3 top red {'stop'}
shirt guy
shirt red
shirt top
shirt tie
hiperonimos sobre sinonimos tie shirt
black guy
black red
black top
black tie
black make
black speech
tie guy
tie red
tie top
tie tie
tie make
hiperonimos sobre sinonimos make tie
s

In [26]:
df_resultados = pd.DataFrame(new_data)
df_resultados

,sumas,distancias,entropia_total,entropias,mutinf,mearts,max_info,list_comp,diferencias,list_incomp,list_M,list_m,list_T,Jaro-Winkler_rit,negT,verbT,negH,verbH,clases
0,0.000000,0.000000,2.337,0.000,0.000000,0.000000,0.000000,5,0.000000,0,5,0,8,0.709596,0,,0,,1
1,1.262909,2.883895,1.664,1.627,4.158247,0.001126,0.892967,1,0.285714,0,3,2,7,0.563889,0,,0,,1
2,0.000000,0.000000,2.000,0.000,0.000000,0.000000,0.000000,4,0.000000,0,4,0,3,0.850000,0,,1,go,1
3,0.000000,0.000000,2.016,0.000,0.000000,0.000000,0.000000,6,0.000000,0,4,0,23,0.364846,0,,0,,1
4,0.000000,0.000000,1.500,0.000,0.000000,0.000000,0.000000,1,0.000000,0,1,0,4,0.722222,0,,0,,1
5,0.000000,0.000000,2.419,0.000,0.000000,0.000000,0.000000,3,0.000000,0,3,0,3,0.666667,0,,1,knock,1
6,0.000000,0.000000,1.411,0.000,0.000000,0.000000,0.000000,1,0.000000,0,1,0,18,0.354167,0,,0,,1
